In [44]:
import pandas as pd

In [45]:
TransactionRecords_All = pd.read_csv('Transaction/Transactions_cointegration.csv', index_col=0)
TransactionRecords_All['Date'] = pd.to_datetime(TransactionRecords_All['Date'])

In [46]:
TransactionRecords_All = TransactionRecords_All.sort_values(['Date', 'Open/Close', 'Period No.', 'Pair No.', 'Round Trip No.', 'Crypto']).reset_index(drop=True)

In [47]:
initialCapital = 100000
pairLimit = 20
capitalPerPair = initialCapital/pairLimit

In [48]:
executedTrades = pd.DataFrame(columns=TransactionRecords_All.columns)
mainOpenTransactionIDs = []
pairOpenTransactionIDs = []

for _, transaction in TransactionRecords_All.iterrows():

    # get current TransactionID
    TransactionID = str(transaction['Period No.']) + " " + str(transaction['Pair No.']) + " " + str(transaction['Round Trip No.'])

    # Close Transaction
    if transaction['Open/Close'] == "Close" and TransactionID in mainOpenTransactionIDs:
        executedTrades.loc[len(executedTrades)] = transaction
        mainOpenTransactionIDs.remove(TransactionID)
    # for the pair transaction record 
    elif transaction['Open/Close'] == "Close" and TransactionID in pairOpenTransactionIDs:
        executedTrades.loc[len(executedTrades)] = transaction
        pairOpenTransactionIDs.remove(TransactionID)

    # Open Transaction record
    if len(mainOpenTransactionIDs) < pairLimit and transaction['Open/Close'] == "Open" and TransactionID not in mainOpenTransactionIDs:
        executedTrades.loc[len(executedTrades)] = transaction
        mainOpenTransactionIDs.append(TransactionID)
    # for the pair transaction record
    elif transaction['Open/Close'] == "Open" and TransactionID in mainOpenTransactionIDs:
        executedTrades.loc[len(executedTrades)] = transaction
        pairOpenTransactionIDs.append(TransactionID)

In [61]:
executedTrades

,Date,Crypto,Long/Short,Price,Open/Close,Transaction pair,Period No.,Pair No.,Round Trip No.,Quantity
0,2019-01-15,CRYPTO:KMDUSD,Long,0.631453,Open,CRYPTO:LINKUSD,1,10,1,1.0
1,2019-01-15,CRYPTO:LINKUSD,Short,0.515329,Open,CRYPTO:KMDUSD,1,10,1,8.711759
2,2019-01-20,CRYPTO:BNTUSD,Long,0.552844,Open,CRYPTO:REPUSD,1,4,1,1.0
3,2019-01-20,CRYPTO:REPUSD,Short,18.207781,Open,CRYPTO:BNTUSD,1,4,1,0.093057
4,2019-01-22,CRYPTO:KMDUSD,Long,0.659547,Open,CRYPTO:LINKUSD,1,2,1,0.083988
...,...,...,...,...,...,...,...,...,...,...
2927,2023-08-29,CRYPTO:MINAUSD,Short,0.398578,Close,CRYPTO:HBARUSD,28,107,1,0.064944
2928,2023-08-29,CRYPTO:APEUSD,Short,1.489468,Close,CRYPTO:UNIUSD,28,114,1,1.0
2929,2023-08-29,CRYPTO:UNIUSD,Long,4.805099,Close,CRYPTO:APEUSD,28,114,1,1.007134
2930,2023-08-29,CRYPTO:BTCUSD,Long,27723.45,Close,CRYPTO:WOOUSD,28,115,1,0.00001


In [62]:
# Scale each trade back to capitalPerPair (5,000) when open the trade


In [63]:
# calcualte daily PnL

In [64]:
# calculate daily return

In [65]:
# comparing with other benchmarks and strategies